<a href="https://colab.research.google.com/github/vshalisko/GEE/blob/main/AUEM/Taller_20241217/ESE_visualizacion_zona_estudio_con_reduccion_nubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensayo 2024.12.17
## Ensenada
### Visuzlización de zona de estudio en Colab - GEE
### Vesión con el proceso de generar mosaico para combinar imagenes de varias zonas y/o para eliminar nubes


In [38]:
## Definir variables

## Prefilo de zona
prefijo = "ESE"

## Punto de centro de zona en coordenadas geográficas
punto_interes_text = [-116.6064, 31.8680]

## Proyeccion UTM aplicable
projection_text = "EPSG:32611"

## limites de rectangulo de zona de estudio en UTM
#limites_utm = [531951, 547066, 3513217, 3532728]    # versión agosto 2024
limites_utm = [523989, 547709, 3503560, 3532709]

## tamaño de pixel
pixel = 30

## número de pixeles máximo en segmento
max_tile = 50000

## fechas para imagenes de satelite
#inicio_2015 = "2015-02-01"
#final_2015 = "2015-06-30"

## datos para definición de rango de fechas imagenes de satelite
## rango de meses
m_inicio = 2
m_final = 6
## rango de años
y_inicio_2015 = 2015
y_final_2015 = 2015

## bandas de L8 utiles
bands_L8 = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']

## bandas de capa de referencia
bands_B = ['settlement']

#### Praparar el entorno

In [ ]:
## cargar bibliotecas GEE y GEEmap
import ee
import geemap

# Iniciar autentificacion
ee.Authenticate()

# Inicializar proyecto GEE
ee.Initialize(project='ee-vshalisko')

#### Preparar datos para GEE

In [39]:
punto_interes = ee.Geometry.Point(punto_interes_text)

projection = ee.Projection(projection_text)

bbox = ee.Geometry.Rectangle([limites_utm[0], limites_utm[2], limites_utm[1], limites_utm[3]], projection, True, False)

#### Definir funciones auxiliares para control de nubes y calidad de pixeles

In [40]:
## funcion para eliminar nubes y sombras
def maskClouds(image):
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = (1 << 3)
    cloud_bit_mask = (1 << 4)
    cloud_mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_bit_mask).eq(0))
    return image.updateMask(cloud_mask)

#### Seleccionar imagen Landsat 8 para el año 2015

In [50]:
## Ejemplo de coleccion con presencia de nubes
L8_collection_2015 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                      .filterBounds(punto_interes)
                      .filter(ee.Filter.calendarRange(m_inicio,m_final,"month"))
                      .filter(ee.Filter.calendarRange(y_inicio_2015, y_final_2015, "year"))
                      .filter(ee.Filter.lessThan("CLOUD_COVER", 30)))
#                      .filter(ee.Filter.lessThan("CLOUD_COVER", 4)))

print('Capas de datos filtrados 2015:', L8_collection_2015.size().getInfo())

# Cargar imagen de la colección (Landsat 8).
L8_image_2015 = L8_collection_2015.first()
image_meta_2015 = L8_image_2015.getInfo()
imagename_2015 = image_meta_2015.get('properties',{}).get('LANDSAT_PRODUCT_ID')
print('Imagen 2015 elegido:', imagename_2015)

## obtener lista de nombres de bandas
bands_original_names = L8_image_2015.bandNames();

## Extraer datos de proyeccion de la imagen L8
projection = L8_image_2015.projection().getInfo();

## Consultar datos de la proyección
print(projection.get('crs'))
print(projection.get('transform'))

## generar el mosaico de 2015
L8_image_2015_mosaic = L8_collection_2015.map(maskClouds).reduce(ee.Reducer.mean())

## renombrar bandas al esquema original
L8_image_2015_mosaic = L8_image_2015_mosaic.rename(bands_original_names)

## asignar proyeccion al mosaico
L8_image_2015_re = L8_image_2015_mosaic.reproject(projection.get('crs'), projection.get('transform'), None)

Capas de datos filtrados 2015: 9
Imagen 2015 elegido: LC08_L2SP_039038_20150214_20200910_02_T1
EPSG:32611
[30, 0, 479985, 0, -30, 3628215]


#### Visualizar datos en GEE

In [51]:
## Parametros de visualizacion del poligono
visPol = {
  'color': '0000FF',
  'width': 2,
  'opacity': 0.25
};

## Parametros de visualizacion para Landsat 8
vizParams_L8 = {
  'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
   'min': 5000,
   'max': 15000
}

## definir ventana de mapa
map = geemap.Map()

## Centrar
map.centerObject(punto_interes, 10)


## Agregar capas de datos al mapa
map.addLayer(L8_image_2015, vizParams_L8, "2015 solo Composición de falso color", True, 0.5)
map.addLayer(L8_image_2015_re, vizParams_L8, '2015 mosaic Composición de falso color', True, 0.5)
map.addLayer(bbox, visPol, "Area de estudio", True, 1)
map.addLayer(punto_interes, {}, "Punto de centro", True, 1)


# Presentar el mapa
map

Map(center=[31.868000000000002, -116.6064], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…